CORRELATIONS INTRODUCING LAG


In [1]:
%matplotlib inline

import cosima_cookbook as cc
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import cartopy.crs as ccrs
import xarray as xr
import cmocean.cm as cmocean
import glob
from matplotlib.colors import ListedColormap

import logging
logging.captureWarnings(True)
logging.getLogger('py.warnings').setLevel(logging.ERROR)

from dask.distributed import Client

figdir = '/g/data/x77/ns3783/'

In [3]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/44531/status,
Dashboard: /proxy/44531/status,Workers: 7
Total threads: 28,Total memory: 125.19 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36755,Workers: 7
Dashboard: /proxy/44531/status,Total threads: 28
Started: Just now,Total memory: 125.19 GiB
Comm: tcp://127.0.0.1:36263,Total threads: 4
Dashboard: /proxy/33803/status,Memory: 17.88 GiB
Nanny: tcp://127.0.0.1:45621,


In [4]:
master_session = cc.database.create_session('/g/data/ik11/databases/cosima_master.db')
iaf_cycle2 = '01deg_jra55v140_iaf_cycle2'
iaf_cycle3 = '01deg_jra55v140_iaf_cycle3'

first_year = '1958'
last_year = '2018'

start_time=first_year+'-01-01'
end_time=last_year+'-12-31'

time_period = str(int(start_time[:4]))+'-'+str(int(end_time[:4]))


Load previously calculated temperature time series in Box 1

In [5]:
# load temperature data at 400m:

depth_slice = slice(400, 6000)
# cut the data BOX 1
lon_slice  = slice(-252.7, -251.7)
lat_slice  = slice(-65.35, -65.1)

temp_400m = cc.querying.getvar(iaf_cycle3,'temp' , master_session,frequency='1 monthly',start_time=start_time, end_time=end_time) - 273.15
temp_400m = temp_400m.sel(yt_ocean=lat_slice).sel(xt_ocean=lon_slice).sel(st_ocean=depth_slice)
temp_400m = temp_400m.mean('st_ocean')

# area weighting:
area_t = cc.querying.getvar('01deg_jra55v13_ryf9091','area_t',master_session, ncfile = 'ocean_grid.nc',n=-1)
area_t = area_t.sel(yt_ocean=lat_slice).sel(xt_ocean=lon_slice)

# sum area:
area_t_sum = area_t.sum('xt_ocean').sum('yt_ocean')

# Apply area weight:
temp_400m = (temp_400m*area_t).sum('xt_ocean').sum('yt_ocean')/area_t_sum

# load data
temp_400m = temp_400m.load()

In [6]:
#the annual mean 
# annual averaging:
temp_400m_annual = temp_400m.groupby('time.year').mean(dim='time')
temp_400m_annual = temp_400m_annual.load()

This will be whatever you are correlating with / regressing against (e.g. winds)

In [8]:
# loading the full dataset first so we can create the other special ones

#first do the extended
lat_slice  = slice(-67,-64)
lon_slice  = slice(-255, -248)

first_year = '1958'
last_year = '2018'

start_time=first_year+'-01-01'
end_time=last_year+'-12-31'

tau_x_con = cc.querying.getvar(iaf_cycle3, 'tau_x', master_session, frequency='1 monthly',start_time=start_time, end_time=end_time).sel(yu_ocean=lat_slice).sel(xu_ocean=lon_slice)
tau_x_con_ext=tau_x_con.mean('yu_ocean').mean('xu_ocean')

tau_y_con = cc.querying.getvar(iaf_cycle3, 'tau_y', master_session, frequency='1 monthly',start_time=start_time, end_time=end_time).sel(yu_ocean=lat_slice).sel(xu_ocean=lon_slice)
tau_y_con_ext=tau_y_con.mean('yu_ocean').mean('xu_ocean')


In [9]:
#load data
tau_x_con_ext = tau_x_con_ext.load()
tau_y_con_ext = tau_y_con_ext.load()

In [10]:
#full, no time averaging
#do the contrained one
lat_slice  = slice(-67,-64)
lon_slice  = slice(-255, -248)

tau_x_con = cc.querying.getvar(iaf_cycle3, 'tau_x', master_session, frequency='1 monthly',start_time=start_time, end_time=end_time).sel(yu_ocean=lat_slice).sel(xu_ocean=lon_slice)
tau_x_con_full=tau_x_con.mean('yu_ocean').mean('xu_ocean')

tau_y_con = cc.querying.getvar(iaf_cycle3, 'tau_y', master_session, frequency='1 monthly',start_time=start_time, end_time=end_time).sel(yu_ocean=lat_slice).sel(xu_ocean=lon_slice)
tau_y_con_full=tau_y_con.mean('yu_ocean').mean('xu_ocean')

#load data
tau_x_con_full = tau_x_con_full.load()
tau_y_con_full = tau_y_con_full.load()

Function for doing spatial correlations/regressions:

This comes from http://hrishichandanpurkar.blogspot.com/2017/09/vectorized-functions-for-correlation.html


In [11]:
def lag_linregress_3D(x, y, lagx=0, lagy=0):
    """
    Input: Two xr.Datarrays of any dimensions with the first dim being time. 
    Thus the input data could be a 1D time series, or for example, have three dimensions (time,lat,lon). 
    Datasets can be provied in any order, but note that the regression slope and intercept will be calculated
    for y with respect to x.
    Output: Covariance, correlation, regression slope and intercept, p-value, and standard error on regression
    between the two datasets along their aligned time dimension.  
    Lag values can be assigned to either of the data, with lagx shifting x, and lagy shifting y, with the specified lag amount. 
    """ 
    #1. Ensure that the data are properly alinged to each other. 
    x,y = xr.align(x,y)
    
    #2. Add lag information if any, and shift the data accordingly
    if lagx!=0:
        #If x lags y by 1, x must be shifted 1 step backwards. 
        #But as the 'zero-th' value is nonexistant, xr assigns it as invalid (nan). Hence it needs to be dropped
        x   = x.shift(time = -lagx).dropna(dim='time')
        #Next important step is to re-align the two datasets so that y adjusts to the changed coordinates of x
        x,y = xr.align(x,y)

    if lagy!=0:
        y   = y.shift(time = -lagy).dropna(dim='time')
        x,y = xr.align(x,y)
 
    #3. Compute data length, mean and standard deviation along time axis for further use: 
    n     = x.shape[0]
    xmean = x.mean(axis=0)
    ymean = y.mean(axis=0)
    xstd  = x.std(axis=0)
    ystd  = y.std(axis=0)
    
    #4. Compute covariance along time axis
    cov   =  np.sum((x - xmean)*(y - ymean), axis=0)/(n)
    
    #5. Compute correlation along time axis
    cor   = cov/(xstd*ystd)
    
    #6. Compute regression slope and intercept:
    slope     = cov/(xstd**2)
    intercept = ymean - xmean*slope  
    
    #7. Compute P-value and standard error
    #Compute t-statistics
    tstats = cor*np.sqrt(n-2)/np.sqrt(1-cor**2)
    stderr = slope/tstats
    
    from scipy.stats import t
    pval   = t.sf(tstats, n-2)*2
    pval   = xr.DataArray(pval, dims=cor.dims, coords=cor.coords)

    return cov,cor,slope,intercept,pval,stderr

# detrending function:
def detrend_dim(da, dim, deg=1):
    # detrend along a single dimension
    p = da.polyfit(dim=dim, deg=deg)
    fit = xr.polyval(da[dim], p.polyfit_coefficients)
    return da - fit

Detrend data and compute correlations / regressions:

In [12]:
# Note we need to first detrend the data:
temp_detrended=detrend_dim(temp_400m,'time')

tau_x_con_ext_detrended=detrend_dim(tau_x_con_ext,'time')
tau_y_con_ext_detrended=detrend_dim(tau_y_con_ext,'time')

tau_x_con_full_detrended=detrend_dim(tau_x_con_full,'time')
tau_y_con_full_detrended=detrend_dim(tau_y_con_full,'time')

In [15]:
#for the extended one 
roll_mean_t_ext = temp_detrended.rolling(time=12, center=True).mean()
roll_mean_x_ext = tau_x_con_ext_detrended.rolling(time=12, center=True).mean()
roll_mean_y_ext = tau_y_con_ext_detrended.rolling(time=12, center=True).mean()

In [19]:
s=11;
for i in range(0,s):
    cov,cor,slope,intercept,pval,stderr = lag_linregress_3D(roll_mean_t_ext,roll_mean_x_ext,
                                                        lagx=i, lagy=0)
    print('lag', i+1, ': ',cor)

lag 1 :  <xarray.DataArray ()>
array(0.24073569)
lag 2 :  <xarray.DataArray ()>
array(0.25345675)
lag 3 :  <xarray.DataArray ()>
array(0.25243444)
lag 4 :  <xarray.DataArray ()>
array(0.24132013)
lag 5 :  <xarray.DataArray ()>
array(0.22234371)
lag 6 :  <xarray.DataArray ()>
array(0.19479783)
lag 7 :  <xarray.DataArray ()>
array(0.16091765)
lag 8 :  <xarray.DataArray ()>
array(0.12468366)
lag 9 :  <xarray.DataArray ()>
array(0.08724571)
lag 10 :  <xarray.DataArray ()>
array(0.04900967)
lag 11 :  <xarray.DataArray ()>
array(0.01218188)


In [20]:
s=11;
for i in range(0,s):
    cov,cor,slope,intercept,pval,stderr = lag_linregress_3D(roll_mean_t_ext,roll_mean_y_ext,
                                                        lagx=i, lagy=0)
    print('lag', i+1, ': ',cor)

lag 1 :  <xarray.DataArray ()>
array(-0.26291728)
lag 2 :  <xarray.DataArray ()>
array(-0.27385188)
lag 3 :  <xarray.DataArray ()>
array(-0.27202085)
lag 4 :  <xarray.DataArray ()>
array(-0.26256257)
lag 5 :  <xarray.DataArray ()>
array(-0.24765891)
lag 6 :  <xarray.DataArray ()>
array(-0.22642602)
lag 7 :  <xarray.DataArray ()>
array(-0.20075626)
lag 8 :  <xarray.DataArray ()>
array(-0.17352428)
lag 9 :  <xarray.DataArray ()>
array(-0.14520387)
lag 10 :  <xarray.DataArray ()>
array(-0.11798707)
lag 11 :  <xarray.DataArray ()>
array(-0.09172917)


In [14]:
#for the full dataset
roll_mean_t = temp_detrended.rolling(time=12, center=True).mean()
roll_mean_x = tau_x_con_full_detrended.rolling(time=12, center=True).mean()
roll_mean_y = tau_y_con_full_detrended.rolling(time=12, center=True).mean()

In [15]:
# This computes everything ZONAL:
#cov,cor,slope,intercept,pval,stderr = lag_linregress_3D(roll_mean_t,roll_mean_x,
                                                        lagx=0, lagy=1)

In [34]:
s=11;
for i in range(0,s):
    cov,cor,slope,intercept,pval,stderr = lag_linregress_3D(roll_mean_t,roll_mean_x,
                                                        lagx=0, lagy=i)
    print('lag', i+1, ': ',cor)

lag 1 :  <xarray.DataArray ()>
array(0.3578452)
lag 2 :  <xarray.DataArray ()>
array(0.37211755)
lag 3 :  <xarray.DataArray ()>
array(0.37135022)
lag 4 :  <xarray.DataArray ()>
array(0.36050524)
lag 5 :  <xarray.DataArray ()>
array(0.34138545)
lag 6 :  <xarray.DataArray ()>
array(0.31417487)
lag 7 :  <xarray.DataArray ()>
array(0.28126298)
lag 8 :  <xarray.DataArray ()>
array(0.24601819)
lag 9 :  <xarray.DataArray ()>
array(0.2095782)
lag 10 :  <xarray.DataArray ()>
array(0.17212192)
lag 11 :  <xarray.DataArray ()>
array(0.13562781)


In [35]:
s=11;
for i in range(0,s):
    cov,cor,slope,intercept,pval,stderr = lag_linregress_3D(roll_mean_t,roll_mean_y,
                                                        lagx=0, lagy=i)
    print('lag', i+1, ': ',cor)

lag 1 :  <xarray.DataArray ()>
array(-0.33875729)
lag 2 :  <xarray.DataArray ()>
array(-0.34857021)
lag 3 :  <xarray.DataArray ()>
array(-0.34432475)
lag 4 :  <xarray.DataArray ()>
array(-0.33164718)
lag 5 :  <xarray.DataArray ()>
array(-0.3123485)
lag 6 :  <xarray.DataArray ()>
array(-0.28586743)
lag 7 :  <xarray.DataArray ()>
array(-0.25456107)
lag 8 :  <xarray.DataArray ()>
array(-0.22175852)
lag 9 :  <xarray.DataArray ()>
array(-0.1880139)
lag 10 :  <xarray.DataArray ()>
array(-0.15558931)
lag 11 :  <xarray.DataArray ()>
array(-0.12451673)
